In [15]:
import soundfile as sf
from tqdm import tqdm
import pandas as pd
import librosa
import json
import os

from jiwer import wer

In [2]:
def load_cached_metadata(filepaths):
    metadata = []
    for filepath in filepaths:
        for line in tqdm(open(filepath).readlines()):
            sample = json.loads(line)
            metadata.append(sample)
            
    return metadata

filepaths = [
    "/data/asr-research/data/s9_stt_metadata_nemo_ctc/predictions_all.jsonl",
]
metadata = load_cached_metadata(filepaths)
metadata = pd.DataFrame(metadata)
metadata.head(1)

100%|██████████| 2231796/2231796 [00:05<00:00, 379724.62it/s]


,audio_filepath,offset,duration,text,pred_text
0,/data2/audio/f88-v1/1082935a-1543-4673-8c46-5d...,1.54,1.69,a a a a a bê,a a a a a bê


In [6]:
metadata["wer"] = metadata.apply(
    lambda row: wer(row["text"], row["pred_text"]), axis=1
)

In [42]:
subset = metadata[metadata.wer == 0]

In [41]:
metadata[metadata.wer == 0].duration.sum() / 3600

np.float64(1629.0257189262998)

In [36]:
row = metadata[metadata.wer > 0.15].iloc[25].to_dict()

audio_filepath = row["audio_filepath"]
start_time = row["offset"]
end_time = start_time + row["duration"]

audio, sr = librosa.load(audio_filepath, sr=None)

audio = audio[int(sr*start_time): int(sr*end_time)]
sf.write("segment.wav", audio, samplerate=sr)

row

{'audio_filepath': '/data2/audio/f88-v1/72ddc3e4-013a-4bfd-bbb6-164e65c17206-0-3-18.17-26.92.wav',
 'offset': 0.54,
 'duration': 1.79,
 'text': 'a lô anh em',
 'pred_text': 'a lô bên em',
 'wer': 0.25}

In [3]:
metadata.duration.sum()/ 3600

np.float64(1939.4076343522306)

In [53]:
vocab = subset.text.apply(lambda x: x.split()).explode().value_counts()

In [78]:
outlier = vocab[vocab < 10]
outlier = set(outlier.index.to_list())

In [74]:
is_valid = subset.text.apply(lambda x: any([word in outlier for word in x.split()]))

In [75]:
subset[is_valid].duration.sum() / 3600

np.float64(22.60478114748663)

In [76]:
subset[~is_valid].duration.sum() / 3600

np.float64(1606.4209377788125)

In [83]:
clean_data = subset[~is_valid]
clean_data = clean_data[
    [
        "audio_filepath", "offset", 
        "duration", "pred_text", "wer"
        ]
    ].rename(columns={"pred_text": "text"})

In [ ]:
filepath = "data/f88_cleaned_data.jsonl"
with open(filepath, "w") as f:
    for index in tqdm(clean_data.index):
        row = clean_data.loc[index].to_dict()
        json_obj = json.dumps(row, ensure_ascii=False)
        f.write(json_obj+"\n")
print(f'saved data to {filepath}')

saved data to data/f88_cleaned_data.jsonl


: 